In [2]:
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Paste your CREATE TABLE and INSERT statements here
cursor.executescript("""
-- Your SQL here
CREATE TABLE CUSTOMERS (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    address VARCHAR(100)
);

CREATE TABLE ITEMS (
    item_id INT PRIMARY KEY,
    item_name VARCHAR(50),
    price DECIMAL(10, 2),
    department VARCHAR(50)
);

CREATE TABLE SALES (
    date DATE,
    order_id INT,
    item_id INT,
    customer_id INT,
    quantity INT,
    revenue DECIMAL(10, 2),
    FOREIGN KEY (item_id) REFERENCES ITEMS(item_id),
    FOREIGN KEY (customer_id) REFERENCES CUSTOMERS(customer_id)
);
"""


)


In [3]:
cursor.executescript("""INSERT INTO CUSTOMERS (customer_id, first_name, last_name, address) VALUES
(1001, 'John', 'Doe', '123 A St'),
(1002, 'Jane', 'Smith', '456 B St'),
(1003, 'Alice', 'Brown', '789 C St'),
(1004, 'Tom', 'Hanks', '321 D St'),
(1005, 'Emma', 'Stone', '654 E St'),
(1006, 'Chris', 'Lee', '987 F St'),
(1007, 'John', 'Doe', '111 G St'),
(1008, 'Mary', 'Moe', '222 H St');""")


In [4]:
cursor.executescript("""INSERT INTO ITEMS (item_id, item_name, price, department) VALUES
(1, 'Pen', 10.00, 'Stationery'),
(2, 'Notebook', 25.00, 'Stationery'),
(3, 'Headphones', 60.00, 'Electronics'),
(4, 'Marker', 15.00, 'Stationery'),
(5, 'Mug', 13.75, 'Home'),
(6, 'USB Cable', 30.00, 'Electronics');

INSERT INTO SALES (date, order_id, item_id, customer_id, quantity, revenue) VALUES
('2023-03-18', 101, 1, 1001, 2, 40.00),
('2023-03-18', 102, 2, 1002, 1, 25.00),
('2023-01-12', 103, 3, 1003, 1, 60.00),
('2023-01-25', 104, 1, 1001, 1, 20.00),
('2022-11-03', 105, 4, 1004, 3, 45.00),
('2022-07-19', 106, 5, 1005, 4, 55.00),
('2022-06-11', 107, 6, 1006, 1, 30.00),
('2023-03-18', 108, 1, 1007, 2, 40.00),
('2023-03-18', 109, 3, 1008, 1, 60.00);


""")

# 1. Total number of orders completed on 18th March 2023

In [6]:
cursor.execute("""
SELECT COUNT(DISTINCT Order_id) AS total_orders
FROM SALES
WHERE Date = '2023-03-18';
""")
result = cursor.fetchone()[0]
print("Total orders on 18th March 2023:", result)


Total orders on 18th March 2023: 4


# 2. Total number of orders completed on 18th March 2023 by 'John Doe'

In [7]:

cursor.execute("""
SELECT COUNT(DISTINCT s.Order_id)
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
WHERE s.Date = '2023-03-18' AND c.First_name = 'John' AND c.Last_name = 'Doe';
""")
print("2. Orders by John Doe on 18th March 2023:", cursor.fetchone()[0])

2. Orders by John Doe on 18th March 2023: 2


# 3. Total number of customers who purchased in Jan 2023 + avg spend per customer

In [8]:

cursor.execute("""
SELECT COUNT(DISTINCT Customer_id),
       ROUND(SUM(Revenue)*1.0 / COUNT(DISTINCT Customer_id), 2) AS avg_spend
FROM SALES
WHERE Date LIKE '2023-01%';
""")
count, avg_spend = cursor.fetchone()
print(f"3. Customers in Jan 2023: {count}, Avg spend per customer: ${avg_spend}")

3. Customers in Jan 2023: 2, Avg spend per customer: $40.0


# 4. Departments that generated less than $600 in 2022


In [9]:
cursor.execute("""
SELECT i.Department, SUM(s.Revenue) AS dept_revenue
FROM SALES s
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE s.Date LIKE '2022%'
GROUP BY i.Department
HAVING dept_revenue < 600;
""")
print("4. Departments with revenue < $600 in 2022:")
for row in cursor.fetchall():
    print(" -", row[0], "($"+str(row[1])+")")

4. Departments with revenue < $600 in 2022:
 - Electronics ($30)
 - Home ($55)
 - Stationery ($45)


# 5. Most and least revenue generated by a single order



In [10]:
cursor.execute("""
SELECT MAX(Revenue), MIN(Revenue)
FROM SALES;
""")
max_rev, min_rev = cursor.fetchone()
print(f"5. Max revenue by order: ${max_rev}, Min revenue: ${min_rev}")

5. Max revenue by order: $60, Min revenue: $20


# 6. What were the orders that were purchased in our most lucrative order.

In [14]:
cursor.execute("""
SELECT *
FROM SALES
WHERE Revenue = (SELECT MAX(Revenue) FROM SALES);
""")

# Print results
rows = cursor.fetchall()
for row in rows:
    print("Order ID:", row[1], "| Date:", row[0], "| Item ID:", row[2],
          "| Customer ID:", row[3], "| Quantity:", row[4], "| Revenue: $", row[5])


Order ID: 103 | Date: 2023-01-12 | Item ID: 3 | Customer ID: 1003 | Quantity: 1 | Revenue: $ 60
Order ID: 109 | Date: 2023-03-18 | Item ID: 3 | Customer ID: 1008 | Quantity: 1 | Revenue: $ 60
